In [1]:
import hoomd
import hoomd.hpmc as hpmc
import coxeter
from ipywidgets import interact, interactive
import ipywidgets as widgets
import plato, plato.draw.vispy as draw
import gsd, gsd.hoomd
%pylab inline

Populating the interactive namespace from numpy and matplotlib


/Users/rachaelskye/opt/anaconda3/lib/python3.8/site-packages/IPython/core/magics/pylab.py:159: UserWarning: pylab import has clobbered these variables: ['draw', 'interactive']
`%matplotlib` prevents importing * from pylab and numpy
  warn("pylab import has clobbered these variables: %s"  % clobbered +


In [2]:
getShape = coxeter.families.RegularNGonFamily()

def make3D(vertex_list):
    to_add = np.transpose([np.zeros(len(vertex_list))])
    vertex_list = np.concatenate([vertex_list,to_add],axis=1)
    return vertex_list

verticesA = make3D(getShape.make_vertices(3))
verticesB = make3D(getShape.make_vertices(3))


In [3]:
verticesA

array([[ 0.87738268,  0.        ,  0.        ],
       [-0.43869134,  0.75983569,  0.        ],
       [-0.43869134, -0.75983569,  0.        ]])

In [4]:
# We can use text input
shape_1 = 'square'

In [5]:
# We can also use more interactive input methods
shape_2 = widgets.Text(
    value='',
    placeholder='Type something',
    description='2nd shape:',
    disabled=False
)

display(shape_2)

Text(value='', description='2nd shape:', placeholder='Type something')

In [6]:
print(shape_2.value) #this is how you access the string for the second shape chosen

In [7]:
# Choose a fraction between 0 and 1
# which will represent the percent of shapes
# that are Shape 1
# shape_ratio = 0 means all the shapes will be Shape 2

shape_ratio = widgets.FloatSlider(
    min=0.00, 
    max=1.00, 
    step=0.01, 
    description='Shape ratio:', 
    value=0.50,
    readout_format='.2f')

display(shape_ratio)

FloatSlider(value=0.5, description='Shape ratio:', max=1.0, step=0.01)

In [8]:
print(shape_ratio.value)

0.5


In [9]:
numParticles_sqrt = 10 # example: if numParticles_sqrt = 10, then there are 100 particles in the system

In [14]:
hoomd.context.initialize("--mode=cpu");

system = hoomd.init.create_lattice(unitcell=hoomd.lattice.unitcell(N=2,
                                                                  a1=[3,0,0],
                                                                  a2=[0,6,0],
                                                                  a3=[0,0,1],
                                                                  dimensions=2,
                                                                  position=[[0,0,0],
                                                                            [0,3,0]],
                                                                  type_name=["A","B"]),
            n=[numParticles_sqrt,int(1/2*numParticles_sqrt)]);

snap = system.take_snapshot()

snap.particles.types = ["A","B"]

for p in range(int(shape_ratio.value*(numParticles_sqrt**2))):
    snap.particles.typeid[p]=0
for p in range(int(shape_ratio.value*(numParticles_sqrt**2)),numParticles_sqrt**2):
    snap.particles.typeid[p]=1
    
system.restore_snapshot(snap)

notice(2): Group "all" created containing 100 particles


In [15]:
#actual simulation setup goes here
mc = hpmc.integrate.convex_polygon(seed=np.random.randint(1,1e6), d=0.5, a=0.5,move_ratio=0.5)
mc.shape_param.set('A', vertices=verticesA)
mc.shape_param.set('B', vertices=verticesB)

boxMC = hpmc.update.boxmc(mc, hoomd.variant.linear_interp(points=[[0,1],[1e5,20]],zero=0), 
                          np.random.randint(1,1e6))

boxMC.volume(delta=1.0, weight=1.0)

particle_tuner = hpmc.util.tune(obj=mc, tunables=['d','a'], target=0.2)
box_tuner = hpmc.util.tune_npt(obj=boxMC, tunables=['dV'],  target=0.2)


all=hoomd.group.all()

In [59]:
traj = hoomd.dump.gsd(filename='traj.gsd',period=int(100),group=all,overwrite=True)
traj.dump_shape(mc)

In [60]:
hoomd.run(1e5)

** starting run **


*Warning*: Box size is too small or move size is too large for the minimum image convention.
Testing 11 images per trial move, performance may slow.
This message will not be repeated.


Time 00:00:10 | Step 17588 / 100000 | TPS 1758.79 | ETA 00:00:46
Time 00:00:20 | Step 34335 / 100000 | TPS 1674.68 | ETA 00:00:39
Time 00:00:30 | Step 42449 / 100000 | TPS 811.327 | ETA 00:01:10
Time 00:00:40 | Step 53868 / 100000 | TPS 1141.86 | ETA 00:00:40
Time 00:00:50 | Step 72146 / 100000 | TPS 1827.7 | ETA 00:00:15
Time 00:01:00 | Step 90317 / 100000 | TPS 1817.07 | ETA 00:00:05
Time 00:01:06 | Step 100000 / 100000 | TPS 1737.31 | ETA 00:00:00
Average TPS: 1524.94
---------
notice(2): -- HPMC stats:
notice(2): Average translate acceptance: 0.122146
notice(2): Average rotate acceptance:    0.252914
notice(2): Trial moves per second:        1.21993e+06
notice(2): Overlap checks per second:     1.88424e+07
notice(2): Overlap checks per trial move: 15.4454
notice(2): Number of overlap errors:      0
notice(2): -- HPMC box change stats:
notice(2): Average volume acceptance: 0.18857
notice(2): Total box changes:        100000
** run complete **


In [61]:
#graph volume over time -- Rachael
#graph packing fraction over time -- Rachael

### Plato visualizations

In [ ]:
def getFrameCount(fname):
    #inputs: fname, the filename (ex: 'dump.gsd')
    #outputs: len(traj), number of frames in simulation
    with gsd.hoomd.open(fname, 'rb') as traj:
        return len(traj)

In [ ]:
prim = draw.Polygons() #how to get it to consider different polygons?
box_prim = draw.Box(color=(0, 0, 0, 1), width=.2)
scene = draw.Scene([prim, box_prim], zoom=.5, clip_scale=5)
scene.show()

filename = 'dump.gsd'
frame_num = getFrameCount(filename)

@ipywidgets.interact(frame_index=range(frame_num))
def plot(frame_index=0):
    with gsd.hoomd.open(filename, 'rb') as traj:
        frame = traj[frame_index]
        box = frame.configuration.box
        for (name, val) in zip(['Lx', 'Ly', 'Lz', 'xy', 'xz', 'yz'], box):
            setattr(box_prim, name, val)
        prim.positions = frame.particles.position
        prim.diameters = np.full(len(frame.particles.position), 1)
    scene.render()